# Etape 1 - Preprocessing du dataset

In [ ]:
# Import librairies
import os
import re
import sys

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from collections import Counter

In [ ]:
# Set paths
path = "."
os.chdir(path)
data_path = path + "\\data"
output_path = path + "\\outputs"
fig_path = path + "\\figures"

In [ ]:
# Suppression des FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [ ]:
# Activation PEP8
%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
# Parametres graphiques
%matplotlib inline
rc = {
    'font.size': 14,
    'font.family': 'Arial',
    'axes.labelsize': 14,
    'legend.fontsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'figure.max_open_warning': 30}

sns.set(font='Arial', rc=rc)
sns.set_style(
    "whitegrid", {
        'axes.edgecolor': 'k',
        'axes.linewidth': 1,
        'axes.grid': True,
        'xtick.major.width': 1,
        'ytick.major.width': 1
        })
sns.set_context(
    "notebook",
    font_scale=1.1,
    rc={"lines.linewidth": 1.5})
pd.set_option('display.max_columns', None)

In [ ]:
# Import des données
filename = "export_sans_dewey.dsv"
with open(os.path.join(data_path, filename), 'r', newline='', encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t',)
    data = list(csv_reader)

In [ ]:
data

In [ ]:
# Define dimension
nb_notice = len(data)
print(f"There are {nb_notice} in this file")

In [ ]:
# Check number of column
len_col = []
for row in data:
    len_col.append(len(row))
max_number_col = max(len_col)
print(f"There are till {max_number_col} in this file")

In [ ]:
# Show distribution
sns.histplot(len_col)

In [ ]:
# Checking and removing badly formatted notices
official_nb_col = 5
bad_formated_notices = [x for x in len_col if x != official_nb_col]
print(f"There are {len(bad_formated_notices)} badly formatted notices")

ids_to_keep = [True if x == official_nb_col else False for x in len_col]
print(f"There are {sum(ids_to_keep)} well formatted rows")

data = [row for row, boo in zip(data, ids_to_keep) if boo]
print(f"Working dataset has {len(data)-1} notices")

In [ ]:
# Creation du fichier de données à vérifier
data_to_check = [row for row, boo in zip(data, ids_to_keep) if not boo]
print(f"Need to check {len(data_to_check)} notices extractions")

data_to_check = pd.DataFrame(data_to_check)
print(data_to_check.shape)
data_to_check.head()

In [ ]:
# Export des données à vérifier
data_to_check.to_csv(
    os.path.join(data_path, "data_to_check_LargeExtraction.csv"),
    index=0)

In [ ]:
# Verification du format des données
df = pd.DataFrame(data[1:], columns=data[0])
print(f"le Fichier de données contient {df.shape[0]} lignes et  {df.shape[1]} colonnes")

In [ ]:
# Visualisation
df.head()

In [ ]:
# Ajout d'une colonne description (Titre + resumé)
df.loc[:, "DESCR"] = df.loc[:, 'TITRE'] + ' ' + df.loc[:, 'RESUME']

# Get RAMEAU labels (vedettes)

In [ ]:
# Mettre à plat TOUS les mots clé
pattern=re.compile(r"[\w;^\s]| -- ")
df["rameau_list_unstack"] = df["RAMEAU"].apply(lambda x: re.split(r';\s*(?![^()]*\))| -- ', x))
df.loc[1:20, ["RAMEAU","rameau_list_unstack"]]

In [ ]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [ ]:
keywords = flatten(df['rameau_list_unstack'])
print(f"There are {len(set(keywords))} different RAMEAU labels (Vedettes)")

In [ ]:
# Répartition des vedettes
print(Counter(keywords))

In [ ]:
from wordcloud import WordCloud
wordcloud = WordCloud(width=1000, height=500, background_color='white').generate_from_frequencies(Counter(keywords))
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)

### Retirer les notices avec les mits clés "ouvrages pour la jeunesse" et "romans pour la jeunesse"

In [ ]:
def check_strings(df, col, string):
    res = []
    res = df[col].apply(lambda x: string in x)
    print(f"Nbre de notices contenant le concept {string} : {sum(res)}")
    return res

In [ ]:
# Check notices
string = "Ouvrages pour la jeunesse"
col = "rameau_list_unstack"
is_string = check_strings(df, col, string)

In [ ]:
# Check notices
string = "Roman pour la jeunesse"
col = "rameau_list_unstack"
is_string2 = check_strings(df, col, string)

In [ ]:
# Reduction du jeu de données
df_reduced = df[(is_string+is_string2) == 0]
print(df_reduced.shape)

In [ ]:
# Wordcloud
keywords2 = flatten(df_reduced['rameau_list_unstack'])
wordcloud2 = WordCloud(
    width=1000, height=500, background_color='white'
    ).generate_from_frequencies(Counter(keywords2))
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud2)

# Clustering avant échantillonnage

In [ ]:
# echantillonnage
nsample = 15000
df_sample = df_reduced.sample(n=nsample).reset_index()
df_sample.shape

In [ ]:
# Selected index
selected_index = df_sample.index
selected_index
selected_ppn = df_sample["PPN"]
selected_ppn

In [ ]:
# Export selected PPN
pd.DataFrame(selected_ppn).to_csv(os.path.join(data_path, "ppn_echantillon_150notices.csv"), index="index")

In [ ]:
# Sauvegarde des titres et résumé des notices
df.loc[selected_index, ["PPN", "TITRE", "RESUME", "RAMEAU"]].to_csv(os.path.join(data_path, "echantillon_150notices.csv"), encoding="utf-8")

In [ ]:
# Encodage des mots clés
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(df_sample["rameau_list_unstack"])
classes = mlb.classes_

In [ ]:
# Check encoding
labels_encoded

In [ ]:
# Check inverse transformation
nlab = 5
labels_true = df_sample.loc[:5, "rameau_list_unstack"]
print(f"True {nlab} first labels : {labels_true}")
print(f"Recoded {nlab} first labels : {mlb.inverse_transform(labels_encoded[:nlab])}")

In [ ]:
# Check classes
nbr = 50
print(f"{classes[:nbr]} premiers mots clés (ordre alphabetique)")
print(f"{classes[-nbr:]} derniers mots clés (ordre alphabetique)")

In [ ]:
# Clustering based on Kmeans
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [ ]:
# test different clusters
sil = []
davis_bouldin = []
sum_of_squared_distances = []
param_range = range(3, 20)
for k in param_range:
    print(f"Clustering with {k} groups")
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(labels_encoded)
    clusters = kmeans.labels_
    sum_of_squared_distances.append(kmeans.inertia_)
    silh = silhouette_score(
        labels_encoded, clusters, metric="euclidean", sample_size=10000, random_state=200
        )
    dav = davies_bouldin_score(labels_encoded, clusters)
    sil.append(silh)
    davis_bouldin.append(dav)

In [ ]:
# Find optimal values accordin to silhouette and Davis-Bouldin scores
opt_val_sil = param_range[sil.index(max(sil))]
opt_val_db = param_range[davis_bouldin.index(min(davis_bouldin))]

In [ ]:
opt_val_sil

In [ ]:
opt_val_db

In [ ]:
def plot_metrics(
    sil, davis_bouldin, param_name, param_range, silhouette_color="red", db_color="blue"
):
    """
    Plot the silhouette score and davies_bouldin score for a range of cluster number.

    Parameters:
    -----------
        - sil (list): list of silhouette scores for all values of hyper-parameter range
        - davis_bouldin (list): list of davies-bouldin scores along hyper-parameter range
        - param_name (str): name of the hyper-parameter to be fined-tuned
        - param_range (list) : range of possible values for hyper-parameter tuning
        - silhouette_color (str): color for the silhouette score (defaut: 'red')
        - db_color (str): color for the silhouette score (defaut: 'blue')

    Returns :
    ---------
        - Evolution of silhouette and Davies-bouldin scores along hyper-parameter range
    """

    # plot
    _, ax1 = plt.subplots()
    ax1.set_xlabel(param_name)
    ax1.set_ylabel("Silhouette_score", color=silhouette_color)
    ax1.plot(param_range, sil, color=silhouette_color)
    ax1.tick_params(axis="y", labelcolor=silhouette_color)

    # Adding Twin Axes
    ax2 = ax1.twinx()
    ax2.set_ylabel("Davies_bouldin", color=db_color)
    ax2.plot(param_range, davis_bouldin, color=db_color)
    ax2.tick_params(axis="y", labelcolor=db_color)

    # Show plot
    plt.show()

In [ ]:
plot_metrics(sil, davis_bouldin, param_name="k", param_range=param_range, silhouette_color="red", db_color="blue")

In [ ]:
# Plot inertia
plt.plot(param_range, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('sum_of_squared_distances')
plt.title('elbow method for optimal k')
plt.show()

In [ ]:
# Best clustering
k = 100
kmeans = KMeans(n_clusters=k)
kmeans.fit(labels_encoded)

In [ ]:
# Graph des silhouettes
from yellowbrick.cluster import SilhouetteVisualizer
print("Graph des Silhouettes\n")
silhouette_vis = SilhouetteVisualizer(kmeans)
silhouette_vis.fit(labels_encoded)
silhouette_vis.poof()

In [ ]:
# Plot clusters
# Fit visualisation pipeline
import sklearn
from sklearn.manifold import TSNE
tsne = TSNE(init="pca").fit_transform(labels_encoded)

In [ ]:
# Plot TSNE
plt.figure(figsize=(10, 10))
axe = plt.axes()
clusters = kmeans.labels_
num_classes = k
palette = np.array(sns.color_palette("tab10", num_classes))
axe.scatter(x=tsne[:, 0], y=tsne[:, 1], c=palette[clusters.astype(int)])

In [ ]:
df_sample["cluster"] = clusters
stratified_sampling = df_sample.sample(n=100, weights=clusters)

In [ ]:
stratified_sampling["cluster"].astype("object").describe(include="all")
stratified_sampling["cluster"].hist(bins=50)

In [ ]:
# Selected index
selected_index = stratified_sampling.index
selected_index
selected_ppn = stratified_sampling["PPN"]
selected_ppn

In [ ]:
# Export selected PPN
pd.DataFrame(selected_ppn).to_csv(os.path.join(data_path, "ppn_echantillon_100notices_stratified_on_50_clusters.csv"), index="index")